In [1]:
from pymoo.termination.default import DefaultMultiObjectiveTermination, DefaultSingleObjectiveTermination
from pymoo.algorithms.moo.nsga2 import NSGA2
from mutation import MyMutation
from crossover import TraceCrossover
from encoder import Encoder
from sampling import MySampling
from callback import UpdatePopulationCallback
from tools import Tools
from Declare4Py.ProcessModels.DeclareModel import DeclareModel
from Declare4Py.D4PyEventLog import D4PyEventLog
import warnings
import random
import pandas as pd
import logging
from terminator import MyTermination
from pymoo.algorithms.soo.nonconvex.ga import GA
from problem import Problem_single_ElementWise, Problem_multi_ElementWise
import time
import itertools
from pymoo.optimize import minimize
from testSetup import Setup
from dataCollector import DataCollector

logging.getLogger('matplotlib').setLevel(logging.WARNING)
warnings.filterwarnings("ignore", ".*feasible.*")

In [2]:
# Test Configuration Lists
pop_list = [2000, 3000]  # population sizes
num_event_list = [20, 30, 50]  # trace lengths
declare_model_list = ["base_model.decl"]  # declare models
use_constraints = ["yes", "no"]
termination_map = {
    "my_termination": lambda pop_size: MyTermination(n_required=int(pop_size * 1.5)), # with the lambda it dynamically takes as a parameter the pop_size
    "multi": lambda _: DefaultMultiObjectiveTermination(
        xtol=1e-8, cvtol=1e-6, ftol=0.0025, period=30, n_max_gen=1000, n_max_evals=100000
    ),
    "single": lambda _: DefaultSingleObjectiveTermination(
        xtol=1e-8, cvtol=1e-6, ftol=1e-6, period=20, n_max_gen=1000, n_max_evals=100000
    ),
}

activities_name = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T"]

In [ ]:
for test_run in range(1, 4):
    file_name = f"results/results_run_{test_run}.csv"
    with open(file_name, "w") as f:
        f.write("ID,Population,TraceLength,Model,Termination,Algorithm, Constraints, ExecutionTime\n")
        ID = 1

        # iterate through all configurations
        for combination in itertools.product(pop_list, num_event_list, declare_model_list, termination_map.keys(), use_constraints):
            pop_size, trace_length, model, termination, constraints= combination

            print(f"Running with ID={ID}: Population={pop_size}, TraceLength={trace_length}, Model={model}, Termination={termination}, Constraints={constraints}")

            # exclude 'my_termination' if 'use_constraint' is "no"
            if constraints == "no" and termination == "my_termination":
                print(f"Skipping combination due to incompatible termination: {termination} with  {constraints} constraints")
                continue

            # initialize shared components and population
            (
                encoder, declare, event_log, dataframe, data_collector
            ) = Setup.initialize_shared_components(
                activities_name=activities_name, path_to_declareModel=f"../declare_models/{model}"
            )
            (
                initial_population, initial_encoded_pop, features_range,
                lower_bounds, upper_bounds, mutation, crossover, sampling
            ) = Setup.setup_initial_population(trace_length, n_traces=10, activities_name=activities_name, encoder=encoder)

            # determine termination and algorithm configurations
            try:
                termination_instance = termination_map[termination](pop_size)
                pop_size_for_terminator = int(pop_size * 1.5) if termination == "my_termination" else pop_size
                algorithm_types = ["single", "multi"] if termination == "my_termination" else [termination]

                # logic to handle constraints
                if constraints == "no": # only run "single" GA algorithm if no constraints
                    algorithm_types = ["single"]
                elif constraints == "yes": # allow both "single" and "multi" GA
                    algorithm_types = ["single", "multi"]

                for algorithm_type in algorithm_types:
                    problem = Setup.create_problem(
                        algorithm_type, trace_length, encoder, declare, initial_encoded_pop,
                        lower_bounds, upper_bounds, event_log, dataframe, constraints
                    )

                     # init callback here because I need the obj data for plots
                    algorithm = Setup.create_algorithm(algorithm_type, problem, pop_size_for_terminator, sampling, crossover, mutation)

                    # run algorithm and logs
                    try:
                        result = minimize(problem, algorithm, termination=termination_instance, verbose=False)
                        exec_time = result.exec_time

                        # save log
                        Setup.log_results(f, ID, pop_size, trace_length, model, termination, algorithm_type,constraints, exec_time)
                        # plot and save progress
                        Setup.plot_and_save_progress(result.algorithm.callback, ID, test_run, algorithm_type, constraints)

                    except Exception as algo_error:
                        Setup.log_results(f, ID, pop_size, trace_length, model, termination, algorithm_type,constraints, error=algo_error)
                    ID += 1
            except Exception as config_error:
                Setup.log_results(f, ID, pop_size, trace_length, model, termination,constraints, "N/A", error=config_error)
                ID += 1
# TODO check if the same file for results and plot exists before creating a new one, otherwise it will be overwritten

Running with ID=1: Population=2000, TraceLength=20, Model=base_model.decl, Termination=my_termination, Constraints=yes
Execution Time (single): 23.55 seconds
Execution Time (multi): 25.97 seconds
Running with ID=3: Population=2000, TraceLength=20, Model=base_model.decl, Termination=my_termination, Constraints=no
Skipping combination due to incompatible termination: my_termination with  no constraints
Running with ID=3: Population=2000, TraceLength=20, Model=base_model.decl, Termination=multi, Constraints=yes
Execution Time (single): 69.27 seconds
Execution Time (multi): 75.76 seconds
Running with ID=5: Population=2000, TraceLength=20, Model=base_model.decl, Termination=multi, Constraints=no
Error encountered with ID=5, Termination=multi, Algorithm=single, Constraints=no: index 0 is out of bounds for axis 1 with size 0
Running with ID=6: Population=2000, TraceLength=20, Model=base_model.decl, Termination=single, Constraints=yes
Execution Time (single): 45.25 seconds
Execution Time (mult